In this notebook, we use [Simon Tatham's work](http://www.chiark.greenend.org.uk/~sgtatham/polyhedra/) on polyhedra as a base to create a blastula-like epithelium (make sure to explore Simon's website, I'll wait here).

### Imports

In [2]:
import numpy as np, pandas as pd
import matplotlib.pylab as plt
%matplotlib inline
from IPython.display import display

from tyssue import config, Epithelium, Sheet
from tyssue import SheetGeometry as sgeom, BulkGeometry as geom
from tyssue.generation import extrude

from tyssue.draw import sheet_view


#### File parser for S. Thatam polyhedra files 

In [3]:
# What to we have?
!ls ../data/polyhedra/

cube			     octahedron			  triakisicosahedron
cuboctahedron		     pentakisdodecahedron	  triakisoctahedron
dodecahedron		     rhombicdodecahedron	  triakistetrahedron
greatrhombicosidodecahedron  rhombictriacontahedron	  truncatedcube
greatrhombicuboctahedron     smallrhombicosidodecahedron  truncateddodecahedron
hexakisicosahedron	     smallrhombicuboctahedron	  truncatedicosahedron
hexakisoctahedron	     tetrahedron		  truncatedoctahedron
icosahedron		     tetrakishexahedron		  truncatedtetrahedron
icosidodecahedron	     trapezoidalhexecontahedron
LICENCE			     trapezoidalicositetrahedron


The file structure is prety strait forward:

In [4]:
%%bash
head ../data/polyhedra/truncatedicosahedron

point A -0.53934466291663152 -0.66666666666666674 -1.4120226591665967
point B -1.0786893258332635 -0.33333333333333326 -1.2060113295832982
point C 0.0 -0.33333333333333348 -1.6180339887498949
point D 0.0 0.33333333333333348 -1.6180339887498949
point E 0.53934466291663152 0.66666666666666674 -1.4120226591665967
point F 1.0786893258332635 0.33333333333333326 -1.2060113295832982
point G 0.0 -0.33333333333333348 1.6180339887498949
point H 0.0 0.33333333333333348 1.6180339887498949
point I -0.53934466291663152 0.66666666666666674 1.4120226591665967
point J -1.0786893258332635 0.33333333333333326 1.2060113295832982


In [5]:
%%bash
tail ../data/polyhedra/truncatedicosahedron

face JBfAdZAr Ad
face JBfAdZAr Z
face JBfAdZAr Ar
normal JBfAdZAr -0.85065080835203999 0.0 0.52573111211913359
face LAvAhAjBh L
face LAvAhAjBh Av
face LAvAhAjBh Ah
face LAvAhAjBh Aj
face LAvAhAjBh Bh
normal LAvAhAjBh 0.85065080835203999 0.0 0.52573111211913359


In [6]:
def parse_polyhedron(fname):
    '''
    Parses a polynome description provided by Simon Tatham
    
    Polyhedron files are retrieved from
    http://www.chiark.greenend.org.uk/~sgtatham/polyhedra/
    
    Parameters
    ----------
    fname: str or path
      The polyhedron file to parse
    
    Returns
    -------
    datasets: dict
      The vert, edge and face datasets


    The software in this archive is copyright 2003,2004 Simon Tatham.

    Permission is hereby granted, free of charge, to any person
    obtaining a copy of this software and associated documentation files
    (the "Software"), to deal in the Software without restriction,
    including without limitation the rights to use, copy, modify, merge,
    publish, distribute, sublicense, and/or sell copies of the Software,
    and to permit persons to whom the Software is furnished to do so,
    subject to the following conditions:

    The above copyright notice and this permission notice shall be
    included in all copies or substantial portions of the Software.

    THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND,
    EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF
    MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND
    NONINFRINGEMENT.  IN NO EVENT SHALL THE COPYRIGHT HOLDERS BE LIABLE
    FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF
    CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION
    WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
    '''
    
    
    with open(fname, 'r+') as th:
        lines = th.readlines()

    point_lines = [l.split(' ') for l in lines if l.startswith('point')]
    # point Bh 1.0786893258332635 0.33333333333333326 1.2060113295832982

    points = np.array([[np.float(s) for s in p[-3:]] for p in point_lines])
    point_ids = [p[1] for p in point_lines]

    vert_df = pd.DataFrame(points, index=pd.Index(point_ids), columns=['x', 'y', 'z'])
    vert_df['int_idx'] = np.arange(vert_df.shape[0])

    face_lines = [l.split(' ')[1:] for l in lines if l.startswith('face')]
    # ABAxAwDC A
    prev = face_lines[0][0] # ABAxAwDC
    faces = []
    cur = []
    for ff, l in face_lines:
        if ff != prev:
            faces.append(cur)
            cur = [l[:-1]]
            prev = ff
        else:
            cur.append(l[:-1])
    
    faces.append(cur)
    
    print(len(faces))

    srces = []
    trgts = []
    face_idxs = []
    for i, ff in enumerate(faces):
        srces.extend(vert_df.loc[ff, 'int_idx'])
        trgts.extend(vert_df.loc[np.roll(ff, 1), 'int_idx'])
        face_idxs.extend(np.ones(len(ff))*i)

    edge_df = pd.DataFrame(np.vstack([srces, trgts, face_idxs]).T.astype(np.int),
                           columns=['srce', 'trgt', 'face'])
    edge_df.index.name = 'edge'
    vert_df['letter'] = vert_df.index
    vert_df.set_index('int_idx', drop=True, inplace=True)
    vert_df.index.name = 'vert'
    face_df = pd.DataFrame(index=pd.Index(range(len(faces)), name='face'),
                           columns=['x', 'y', 'z'])

    datasets = {'vert': vert_df,
                'edge': edge_df,
                'face': face_df}
    return datasets


## Truncated Icosahedron (or fullerene)

In [7]:
apical_datasets = parse_polyhedron('../data/polyhedra/truncatedicosahedron')
specs = config.geometry.spherical_sheet()

sheet = Sheet('sphere', apical_datasets, specs)
mod_specs = config.dynamics.quasistatic_sheet_spec()
sheet.update_specs(mod_specs)
sgeom.update_all(sheet)
#res = solver.find_energy_min(sheet, sgeom, model)
#print(res['success'])

32


In [12]:
fig, mesh = sheet_view(
    sheet,
    mode="3D",
    face={
        "visible": True,
        "color": sheet.face_df.area
    }
)
fig

Figure(camera=PerspectiveCamera(aspect=0.8, fov=46.0, matrixWorldNeedsUpdate=True, position=(-1.55918319247641…

In [13]:
from tyssue.io import hdf5
hdf5.save_datasets('../data/hf5/fullerene.hf5', sheet)

### Creating a monolayer through extrusion

In [14]:
apical_datasets = parse_polyhedron('../data/polyhedra/truncatedicosahedron')
datasets = extrude(apical_datasets, scale=0.8)
specs = config.geometry.bulk_spec()

eptm = Epithelium('truncatedicosahedron', datasets, specs)
geom.update_all(eptm)

32


/home/guillaume/miniconda3/envs/py37/lib/python3.7/site-packages/tyssue-0.3.3-py3.7-linux-x86_64.egg/tyssue/generation/modifiers.py:130: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  datasets["face"] = pd.concat([apical_face, basal_face, lateral_face])


In [15]:
fig, mesh = sheet_view(
    eptm,
    mode="3D",
    face={
        "visible": True,
        "color": eptm.face_df.area
    }
)
fig

/home/guillaume/miniconda3/envs/py37/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


Figure(camera=PerspectiveCamera(aspect=0.8, fov=46.0, matrixWorldNeedsUpdate=True, position=(-1.55918319247641…

#### Try out others! 

In [19]:
import ipyvolume as ipv

In [20]:
apical_datasets = parse_polyhedron(
    '../data/polyhedra/trapezoidalicositetrahedron')
specs = config.geometry.spherical_sheet()

sheet = Sheet('sphere', apical_datasets, specs)
mod_specs = config.dynamics.quasistatic_sheet_spec()
sheet.update_specs(mod_specs)
sgeom.update_all(sheet)


ipv.clear()
fig, mesh = sheet_view(
    sheet,
    mode="3D",
    face={
        "visible": True,
        "color": sheet.face_df.area
    }
)
fig

24


/home/guillaume/miniconda3/envs/py37/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/guillaume/miniconda3/envs/py37/lib/python3.7/site-packages/tyssue-0.3.3-py3.7-linux-x86_64.egg/tyssue/draw/ipv_draw.py:205: UserWarning: Attempting to draw a colormap with a uniform value
  warnings.warn("Attempting to draw a colormap " "with a uniform value")


Figure(camera=PerspectiveCamera(fov=46.0, position=(0.0, 0.0, 2.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.…

In [21]:
apical_datasets = parse_polyhedron(
    '../data/polyhedra/greatrhombicosidodecahedron')
specs = config.geometry.spherical_sheet()

sheet = Sheet('sphere', apical_datasets, specs)
mod_specs = config.dynamics.quasistatic_sheet_spec()
sheet.update_specs(mod_specs)
sgeom.update_all(sheet)

ipv.clear()
fig, mesh = sheet_view(
    sheet,
    mode="3D",
    face={
        "visible": True,
        "color": sheet.face_df.area
    }
)
fig

62


Figure(camera=PerspectiveCamera(fov=46.0, position=(0.0, 0.0, 2.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.…